In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/sentiment data/tr_sen_fin.csv')

In [ ]:
df = df.drop(['sentiment', 'kfold','len'],axis = 1)

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-21 06:37:17--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-21 06:37:18--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.56K  --.-KB/s    in 0s      

2021-04-21 06:37:18 (34.9 MB/s) - written to stdout [1594/1594]

setup Colab 

In [ ]:
# from sklearn.model_selection import train_test_split
import pyspark.sql.functions as F
from pyspark.sql.types import StringType,StructType,StructField,IntegerType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import VectorUDT, DenseVector
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import time
prog_start_time = time.time()
import json
# sent_small_bert_L12_128
# spark = ss.builder.master("local[*]").getOrCreate()
spark = sparknlp.start() 
schema_sarc = StructType([StructField('id', StringType(), True),                     
                      StructField('text', StringType(), True),
                      StructField('label', StringType(), True)])


# df_pan = pd.read_csv('/Users/ishansharma/Desktop/train_sarc_final.csv')
# df_src = df_pan[df_pan['sarcasm']=='sarcasm'].sample(n=500)
# df_nsrc = df_pan[df_pan['sarcasm']=='normal'].sample(n=500)

# df_pan = pd.concat([df_src,df_nsrc],ignore_index=True)
# df_pan = df_pan.sample(frac=1).reset_index(drop=True)
# d = {'sarcasm':1,'normal':0}
# df_pan['label'] = df_pan['sarcasm'].apply(lambda x: d[x])
# df_pan = df_pan.drop('sarcasm',axis = 1)
# x_tr,x_ts,y_tr,y_ts = train_test_split(df_pan.drop('label',axis = 1),df_pan['label'],test_size=0.25,stratify=df_pan['label'])
# # print(x_tr.columns)
# x_tr['label'] = y_tr
# x_ts['label'] = y_ts 
df_tr = spark.createDataFrame(df,schema_sarc)
# df_ts = spark.createDataFrame(x_ts,schema_sarc)
# 
# # df = spark.read.option("header", True).option("inferSchema",True).csv("/Users/ishansharma/Desktop/NLP/Model Selection/Imbalanced problem/train_sent")
# # df = df.withColumnRenamed("preprocessed","text")
document = DocumentAssembler().setInputCol("text").setOutputCol("document")
#use = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("USE_pre_text_all_upd_embed")
# tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
# embeddings = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
#       .setInputCols("sentence") \
#       .setOutputCol("sentence_embeddings")
# spellModel = ContextSpellCheckerModel.pretrained().setInputCols("token").setOutputCol("spell")
# # lemmatizer = LemmatizerModel.pretrained(name="lemma_antbnc").setInputCols(["spell"]).setOutputCol("lemma").setLazyAnnotator(False)
sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
# use = UniversalSentenceEncoder.pretrained().setInputCols(["sentence","spell"]).setOutputCol("embeddings")
embeddings = BertSentenceEmbeddings.pretrained("sent_electra_small_uncased", "en").setInputCols("sentence").setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols("embeddings").setOutputCols("embeddings_vectors").setCleanAnnotations(True).setOutputAsVector(True)
# finisher = Finisher().setInputCols(["token"]).setIncludeMetadata(False)



# # # bert = BertSentenceEmbeddings.pretrained("sent_electra_small_uncased", "en").setInputCols("sentence").setOutputCol("sentence_embeddings")
# # # # use_clf_pipeline = Pipeline(
# # #     stages = [
# # #         document,
# # #         use
# # #     ])
# ml = LogisticRegression(featuresCol='input',labelCol='label')
lemm_pipeline = Pipeline(
    stages = [
        document,
#         tokenizer,
#         # finisher
#         spellModel,
# #        #  lemmatizer,
        sentenceDetector,
        embeddings,
		embeddings_finisher
#         # spell_checker,
#         lemmatizer
    ])



df_lem = lemm_pipeline.fit(df_tr).transform(df_tr)
# df.printSchema() 
# ml_pipeline = Pipeline(stages = [ml])
# df_emb_tr = lemm_pipeline.fit(df_tr).transform(df_tr)
# df_emb_ts = lemm_pipeline.fit(df_ts).transform(df_ts)
df_emb = df_lem.select('id',F.explode('embeddings_vectors').alias('input'),'label')
# df_emb_ts = df.select('id',F.explode('embeddings_vectors').alias('input'),'label')
# 
# model = ml_pipeline.fit(df_emb_tr)
# df_res = model.transform(df_emb_tr.drop('label'))
# df_eval = df_res.join(df_emb_ts,on='id',how='inner').select('label','prediction')
# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
# 
# acc = evaluator.evaluate(df_eval, {evaluator.metricName: "accuracy"})
# f1 = evaluator.evaluate(df_eval, {evaluator.metricName: "f1"})
# weightedPrecision = evaluator.evaluate(df_eval, {evaluator.metricName: "weightedPrecision"})
# weightedRecall = evaluator.evaluate(df_eval, {evaluator.metricName: "weightedRecall"})
# # 
# # 
# smry = {'accuracy': acc,'f1':f1,'weighted precision': weightedPrecision,'weighted recall':weightedRecall}
# print(smry)
# # 
# # 
# 
# 
# 
# # print("TRANSFORMATIONS DONEEEEE!!!!! \n")
# # # df_lem.printSchema()
ud_f = F.udf(lambda r : DenseVector(r),F.StringType())
# # res_df = df_lem.select("u_id",F.explode("embeddings").alias("embeddings_v"))
# # res_df = df_lem.select("id","embeddings_vectors.embeddings")
res_df = df_emb.withColumn("ELEC_embed",ud_f(F.col("input")))
res_df = res_df.drop("input")
# print("SAVING !!!!")
# df_lem.printSchema()
# # conv_start_time = time.time()
# # df_lem = df_lem.select("u_id","embeddings_vectors","sarcasm")
# # df_lem = df_lem.withColumn("embeddings_vectors",df_lem.embeddings_vectors.cast('string'))
# # df_lem.write.option('header','true').csv('/Users/ishansharma/Desktop/NLP/sarc_embed_final')
# # toPandas().to_csv('/Users/ishansharma/Desktop/NLP/Model Selection/Imbalanced problem/train_sent_elec_embed/test.csv',index = False,header = True)
# # print("CONVERSION END TIME: %d, PROGRAM END TIME: %d"%(time.time()-conv_start_time,time.time()-prog_start_time))

sent_electra_small_uncased download started this may take some time.
Approximate size to download 48.7 MB
[OK!]


In [ ]:
res_df

DataFrame[id: string, label: string, ELEC_embed: string]

In [ ]:
# pred = res_df.withColumn("embeddings",res_df["ELEC_embed"].cast('string'))
# pred = pred.drop('ELEC_embed')
# pred.write.mode("overwrite").option('header','true').csv('/content/sent_ekec_690k')
res_df.write.option('header','true').csv('/content/sent_elec_690k')

KeyboardInterrupt: ignored

In [ ]:
!zip -r sent_elec_690k.zip /content/sent_elec_690k/

In [ ]:
!cp sent_elec_690k.zip /content/drive/MyDrive/sent_elec_690k.zip